<a href="https://colab.research.google.com/github/cappelchi/ZC/blob/master/cappelchi_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def short_label_equity(df, TakeUp, TrailDown, setY = False):
    #warnings.simplefilter(action='ignore', category=FutureWarning)
    # df - DataFrame with OHLC, Open, High, Low, Close
    # TakeUp - Take profit in form 1 + (% expected profit) / 100
    # TrailDown - Max Drawdown in form 1 +(% max drawdown) / 100
    # setY set to True if need labaling
    # return:
    # i -  bars calculated
    # Trades - max potential trades counted 
    # Profit - max potential profit calculated
    # Y - pd.series with labels for prediction
    
    CurLow = Low1 = df.at[0, 'Low']
    CurHigh = High1 = High2 = df.at[0, 'High']
    LowChange = HighChange = High2Chang = 0
    Flag = LowI = HighI = 0
    HPrI = LPrI = 0
    HMark = np.zeros (np.shape(df)[0])
    HMark2 = np.zeros (np.shape(df)[0])
    LMark = np.zeros (np.shape(df)[0])
    Y = pd.Series(df.index)
    Y[:] = 0
    Profit = Trades = 0
        
    for i in range(1, np.shape(df)[0]):
        CurLow = df.at[i, 'Low']
        CurHigh = df.at[i, 'High']
        if CurLow < Low1:
            LowChange = 1
        if CurHigh > High1:
            HighChange = 1
        if CurHigh > High2:
            High2Change = 1     
        if not Flag:
            if LowChange:
                Low1 = CurLow
                if High1 / Low1 > TakeUp:
                    LowI = i
                    LPrI = CurLow
                    High2 = CurHigh
                    Flag = 2
                elif High1 /Low1 > TrailDown:
                    High2 = Low1 * TrailDown
                    Flag = 1
            
            if HighChange:
                High1 = HPrI = CurHigh
                HighI = i
                if High1 / Low1 > TrailDown:
                    Low1 = CurLow
        elif Flag == 1:                    
            if LowChange:
                Low1 = CurLow
                High2 = Low1 * TrailDown
                if High1 / Low1 > TakeUp:
                    LowI = i
                    LPrI = CurLow
                    High2 = CurHigh
                    Flag = 2
                    
            if HighChange:
                High = CurHigh
                if High1 / Low1 > TrailDown:
                    HighI = i
                    HPrI = High1 = High2 = CurHigh
                    LowI = CurLow
                    Flag = 0
            if High2Change:
                High2 = CurHigh
                if High2 / Low1 > TrailDown:
                    HMark2[i] = High2
                    Low1 = LPrI = CurLow
                    High1 = High2 = HPrI = CurHigh
                    LowI = HighI = i
                    Flag = 0
                    
        elif Flag == 2:
            if LowChange:
                Low1 = CurLow
                if High1 / Low1 > TakeUp:
                    LowI = i
                    LPrI = CurLow
                    High2 = CurHigh
            
            if High2Change:
                High2 = CurHigh
                if High2 / Low1 > TrailDown:
                    LMark[LowI] = LPrI
                    HMark[HighI] = HPrI
                    HMark2[i] = High2
                    Profit = Profit + (HPrI - LPrI)
                    Trades = Trades + 1
                    if setY:
                        for j in range(HighI, LowI + 1):
                            Y[j] = 1
                    High1 = High2 = HPrI = CurHigh
                    Low1 = LPrI = CurLow
                    LowI = HighI = i
                    Flag = 0
        High2Change = 0
        HighChange = 0
        LowChange = 0
                    
                    
    return i, Profit, Trades, Y

TakeUp = 1.098 #@param {type:"number"}
TrailDown = 1.025 #@param {type:"number"}
Labeling = True #@param ["True", "False"] {type:"raw"}
Bars, Prft, Trds, Y = short_label_equity(df, TakeUp, TrailDown, setY = Labeling)
print()
print('Результаты разметки')
print (f'TakeUp = {TakeUp}, TrailDown = {TrailDown}')
print(f'Кол-во бар просмотрено = {Bars}, Прибыль на 1$ входа = {Prft}')
print (f'Потенциальных сделок = {Trds}, Размечено бар = {Y.sum()}')                            
df['Y'] = pd.Series(Y.values, index = df.index) 

In [0]:
#@title Параметры разметки { vertical-output: true }
def TFSelect (TakeUp, TrailDown, df, setY = False):
    #Функция размечает только шорты
    
    # СТАРАЯ ВЕРСИЯ
    
    LArr = np.shape(df.Open)
    HMark = np.zeros (LArr)    # массивы в котором сохраняются Тейковые хаи в позицию бара (размер массива = кол-ву баров), остальные значения  = 0
    LMark = np.zeros (LArr)    # ---||----
    HMark2 = np.zeros (LArr)   # ---||----
    Y = pd.Series(df.index)
    
    Y[:] = 0
    HighChange = 0  # Флаг появления локального максимума в текущем баре
    LowChange = 0   # Флаг появления локального минимума в текущем баре
    Low2Change = 0  # Флаг наличия траэйл просадки от локального максимуму, флаг на сброс
    High2Change = 0 # Флаг наличия траэйл просадки от локального максимума, флаг на сброс,  если разметка в лонг
    HPrI = 0        # Запоминаем котировку последнего локального максимма
    HighI = 0       #(номер бара последнего локально максимума)
    LPrI = 0        # Запоминаем котировку последнего локального минимума
    LowI = 0        # (номер бара последнего локально минимума)
    Flag = 0        # Флаг состояния, 0- начальное состояние, 1 - текущая прибыль прошла барьер максимальной просадки (нужно, т.к. если просадка большая переходим в состояние 1)
                    # Текущая прибыль достигла минимального тейка
    Profit = 0
    Trades = 0
    Low1 = df.Low[0]
    Low2 =df.Low[0]
    High1 = df.High[0]
    High2 = df.High[0]
    #dtind = df.index[0]

    for count, op in tqdm(enumerate(df['Open'])):
    #for count in range(0, 1000):
        #pdb.set_trace()
        
        if df.Low[count] < Low1:
            LowChange = 1
            #pdb.set_trace()
        if df.High[count] > High1:
            HighChange = 1
            #pdb.set_trace()
        if df.High[count] >High2:
            High2Change = 1
            #pdb.set_trace()
        if not Flag:
            if LowChange:
                Low1 = df.Low[count]
                if High1 / Low1 > TakeUp:
                    #print (f'point1: Low1 = {Low1}, High1/Low1 = {High1/Low1} \
                    #TakeUp = {TakeUp}, Count = {count}')
                    LowI = count
                    LPrI = df.Low[count]
                    High2 = df.High[count]
                    Flag = 2
                elif High1 / Low1 > TrailDown: # активируем проверку на просадку, после прохода на % просадки                    
                    #High2 = df.High[count]
                    High2 = Low1 * TrailDown
                    Flag = 1
                    
            if HighChange:
                High1 = df.High[count]
                HighI = count
                HPrI = df.High[count]
                if High1 / Low1 > TrailDown:
                    Low1 = df.Low[count]
            
        elif Flag == 1:
            if LowChange:
                Low1 = df.Low[count]
                ########
                High2 = Low1 * TrailDown
                if High1 / Low1 > TakeUp:
                    #print('point2')
                    LowI = count
                    LPrI = df.Low[count]
                    High2 = df.High[count]
                    Flag = 2
                    
            if HighChange:
                High1 = df.High[count]
                if High1 / Low1 > TrailDown:
                    HighI = count
                    HPrI = df.High[count]
                    Low1 = df.Low[count]
                    High1 = df.High[count]
                    High2 = df.High[count]
                    Flag = 0
            if High2Change:
                High2 = df.High[count]
                if High2 / Low1 > TrailDown:                       
                    Low1 = df.Low[count]
                    High1 = df.High[count]
                    High2 = df.High[count]
                    HMark2[count] = High2                    
                    Flag = 0
                    #######
                    LPrI = df.Low[count]
                    HPrI = df.High[count]
                    LowI = count
                    HighI = count
                    
        elif Flag == 2:
            if LowChange:
                Low1 = df.Low[count]
                if High1 / Low1 > TakeUp:
                    #print (f'point3: Low1 = {Low1}, High1/Low1 = {High1/Low1} \
                    #TakeUp = {TakeUp}, Count = {count}')
                    LowI = count
                    LPrI = df.Low[count]
                    High2 = df.High[count]
                        
            if High2Change:
                High2 = df.High[count]
                if High2 / Low1 > TrailDown:
                    #print (f'point3D: High2 = {High2}, High2/Low1 = {High2/Low1} \
                    #TrailDown = {TrailDown}')
                    LMark[LowI] = LPrI
                    HMark[HighI] = HPrI
                    HMark2[count] = High2
                    Profit = Profit + (HPrI - LPrI)
                    Trades = Trades + 1
                    if setY:                       
                        for i in range(HighI, LowI+1):
                            Y[i] = 1
                       
                    High1 = df.High[count]
                    Low1 = df.Low[count]
                    High2 = df.High[count]
                    LowI = count
                    LPrI = df.Low[count]
                    HighI = count
                    HPrI = df.High[count]
                    Flag = 0
                        
        High2Change = 0
        HighChange = 0
        LowChange = 0
    return count, Profit, Trades, Y

TakeUp = 1.098 #@param {type:"number"}
TrailDown = 1.025 #@param {type:"number"}
Labeling = True #@param ["True", "False"] {type:"raw"}
Bars, Prft, Trds, Y = TFSelect(TakeUp, TrailDown, df, setY = Labeling)
print()
print('Результаты разметки')
print (f'TakeUp = {TakeUp}, TrailDown = {TrailDown}')
print(f'Кол-во бар просмотрено = {Bars}, Прибыль на 1$ входа = {Prft}')
print (f'Потенциальных сделок = {Trds}, Размечено бар = {Y.sum()}')                            
df['Y'] = pd.Series(Y.values, index = df.index) 
df.head()